In [ ]:
import matplotlib.pylab as plt
import numpy as np
import holoviews as hv

from holoviews import opts
hv.extension('bokeh', 'matplotlib')

from landlab import RasterModelGrid, imshow_grid
from landlab.components import FlowAccumulator, LinearDiffuser, FastscapeEroder, ChannelProfiler

In [ ]:
#grid = HexModelGrid((20, 10), 50)

nr = 50
nc = 60
dx = 50

np.random.seed(42)
grid1 = RasterModelGrid((nr, nc), dx)
grid2 = RasterModelGrid((nr, nc), dx)
grid3 = RasterModelGrid((nr, nc), dx)

initial_noise = np.random.rand(grid1.core_nodes.size)

z1 = grid1.add_zeros("topographic__elevation", at="node")
z1[grid1.core_nodes] += initial_noise

z2 = grid2.add_zeros("topographic__elevation", at="node")
z2[grid2.core_nodes] += initial_noise

z3 = grid3.add_zeros("topographic__elevation", at="node")
z3[grid3.core_nodes] += initial_noise

In [ ]:
# uplift, diffusivity
U = 0.001
D = 0.1

# range of K and max slope for slp-K linear scaling
K_min = 0.0001
K_max = 0.0003
K_mid = (K_min + K_max) / 2.
max_slp = 0.2

# timestep
total_time = 1e5
dt = 200
nts = int(total_time / dt)

In [ ]:
K1 = grid1.add_ones("water_erodibility", at="node")
K1[:] = K_mid

K2 = grid2.add_ones("water_erodibility", at="node")
K2[:] = K_mid

K3 = grid3.add_ones("water_erodibility", at="node")
K3[:] = K_mid

In [ ]:
fa1 = FlowAccumulator(grid1)
sp1 = FastscapeEroder(grid1, K_sp="water_erodibility")
ld1 = LinearDiffuser(grid1, linear_diffusivity=D)

fa2 = FlowAccumulator(grid2)
sp2 = FastscapeEroder(grid2, K_sp="water_erodibility")
ld2 = LinearDiffuser(grid2, linear_diffusivity=D)

fa3 = FlowAccumulator(grid3)
sp3 = FastscapeEroder(grid3, K_sp="water_erodibility")
ld3 = LinearDiffuser(grid3, linear_diffusivity=D)

In [ ]:
z1_out = np.empty((nts, grid1.shape[0], grid1.shape[1]))
z2_out = np.empty((nts, grid2.shape[0], grid2.shape[1]))
z3_out = np.empty((nts, grid3.shape[0], grid3.shape[1]))

for i in range(nts):

    # run first half with uplift of U 
    if i > nts / 2:
        factor = 2
    else:
        factor = 1
        
    # uplift
    z1[grid1.core_nodes] += U * dt * factor
    z2[grid2.core_nodes] += U * dt * factor
    z3[grid3.core_nodes] += U * dt * factor

    # topographic__steepest_slope is update in fa.r1s()
    fa1.run_one_step()
    fa2.run_one_step()
    fa3.run_one_step()

    # now we want to update K.
    # say that a waterfall is anything above max_slp
    # When slope is 0 lets make K = K_min
    # When slope is max_slp or above lets make K = K_max
    # We could make K continue to increase, but it might get way too
    # fast... so we'll threshold it for the moment

    slp2 = grid2.at_node["topographic__steepest_slope"].copy()
    slp2[slp2 > max_slp] = max_slp

    grid2.at_node["water_erodibility"][:] = K_mid + (K_max -
                                                     K_mid) * slp2 / max_slp

    # we'll also make K highest when slopes are shallowest and lowest when steepest.
    slp3 = grid3.at_node["topographic__steepest_slope"].copy()
    slp3[slp3 > max_slp] = max_slp

    grid3.at_node["water_erodibility"][:] = K_mid + (K_min -
                                                     K_mid) * slp3 / max_slp

    # run stream power and diffusion.
    sp1.run_one_step(dt)
    ld1.run_one_step(dt)

    sp2.run_one_step(dt)
    ld2.run_one_step(dt)

    sp3.run_one_step(dt)
    ld3.run_one_step(dt)

    z1_out[i, :, :] = z1.reshape(grid1.shape)
    z2_out[i, :, :] = z2.reshape(grid2.shape)
    z3_out[i, :, :] = z3.reshape(grid3.shape)

In [ ]:
plt.plot(grid1.at_node["topographic__steepest_slope"][grid1.core_nodes],
         grid1.at_node["water_erodibility"][grid1.core_nodes],
         ".",
         label="baseline")
plt.plot(grid2.at_node["topographic__steepest_slope"][grid2.core_nodes],
         grid2.at_node["water_erodibility"][grid2.core_nodes],
         ".",
         label="with increase in K with slope")
plt.plot(grid3.at_node["topographic__steepest_slope"][grid2.core_nodes],
         grid3.at_node["water_erodibility"][grid2.core_nodes],
         ".",
         label="with decrease in K with slope")
plt.legend()
plt.xlabel("Slope")
plt.ylabel("K")

In [ ]:
elev_max = max(z1.max(), z2.max(), z3.max())

plt.figure()
imshow_grid(grid1,
            z1,
            plot_name="baseline",
            vmin=0,
            vmax=elev_max,
            cmap='gist_earth')
plt.figure()
imshow_grid(grid2,
            z2,
            plot_name="with increase in K with slope",
            vmin=0,
            vmax=elev_max,
            cmap='gist_earth')

plt.figure()
imshow_grid(grid3,
            z3,
            plot_name="with decrease in K with slope",
            vmin=0,
            vmax=elev_max,
            cmap='gist_earth')

In [ ]:
x = np.arange(grid1.shape[0])
y = np.arange(grid1.shape[1])
t = dt * np.arange(nts)

ds1 = hv.Dataset((t, x, y, z1_out.T), ['Time', 'x', 'y'], 'Elevation')
ds2 = hv.Dataset((t, x, y, z2_out.T), ['Time', 'x', 'y'], 'Elevation')
ds3 = hv.Dataset((t, x, y, z3_out.T), ['Time', 'x', 'y'], 'Elevation')

im1 = ds1.to(hv.Image, ['x', 'y'], label='baseline')
im2 = ds2.to(hv.Image, ['x', 'y'], label="with increase in K with slope")
im3 = ds3.to(hv.Image, ['x', 'y'], label="with decrease in K with slope")

In [ ]:
opts.defaults(
    opts.GridSpace(shared_xaxis=True, shared_yaxis=True),
    opts.Image(cmap='gist_earth', width=400, height=400),
    opts.Labels(text_color='white',
                text_font_size='8pt',
                text_align='left',
                text_baseline='bottom'), opts.Path(color='white'),
    opts.Spread(width=600), opts.Overlay(show_legend=False))

In [ ]:
layout = hv.Layout(im1 + im2 + im3).cols(1)
layout

In [ ]:
slp_vmax = max(grid1.at_node["topographic__steepest_slope"].max(),
               grid2.at_node["topographic__steepest_slope"].max(),
               grid3.at_node["topographic__steepest_slope"].max())

plt.figure()
imshow_grid(grid1,
            "topographic__steepest_slope",
            plot_name="baseline",
            vmin=0,
            vmax=slp_vmax)
plt.figure()
imshow_grid(grid2,
            "topographic__steepest_slope",
            plot_name="with increase in K with slope",
            vmin=0,
            vmax=slp_vmax)
plt.figure()
imshow_grid(grid3,
            "topographic__steepest_slope",
            plot_name="with decrease in K with slope",
            vmin=0,
            vmax=slp_vmax)

In [ ]:
cp1 = ChannelProfiler(grid1, number_of_watersheds=4, main_channel_only=True)
cp1.run_one_step()

cp2 = ChannelProfiler(grid2, number_of_watersheds=4, main_channel_only=True)
cp2.run_one_step()

cp3 = ChannelProfiler(grid3, number_of_watersheds=4, main_channel_only=True)
cp3.run_one_step()

fig, ax = plt.subplots(nrows=2, ncols=3)

plt.axes(ax[0, 0])
cp1.plot_profiles()
ax[0, 0].set_title("baseline")
ax[0, 0].set_ylim([0, elev_max])

plt.axes(ax[0, 1])
cp2.plot_profiles()
ax[0, 1].set_title("with increase in K with slope")
ax[0, 1].set_ylim([0, elev_max])

plt.axes(ax[0, 2])
cp2.plot_profiles()
ax[0, 1].set_title("with decrease in K with slope")
ax[0, 1].set_ylim([0, elev_max])

plt.axes(ax[1, 0])
cp1.plot_profiles_in_map_view()

plt.axes(ax[1, 1])
cp2.plot_profiles_in_map_view()

plt.axes(ax[1, 2])
cp3.plot_profiles_in_map_view()

In [ ]:
plt.loglog(grid1.at_node["topographic__steepest_slope"],
           grid1.at_node["drainage_area"],
           ".",
           label="baseline")
plt.loglog(grid2.at_node["topographic__steepest_slope"],
           grid2.at_node["drainage_area"],
           ".",
           label="with increase in K with slope")
plt.loglog(grid3.at_node["topographic__steepest_slope"],
           grid3.at_node["drainage_area"],
           ".",
           label="with decrease in K with slope")
plt.legend()
plt.ylabel("Slope")
plt.xlabel("Area")